## In this file, we focus on training different models with hyper-parameter tuning

The file is created on Colab to use the GPU device for XGBoost regressor and CatBoost regressor. You can simply ignore the set up.

In [ ]:
import os

In [ ]:
!mkdir /content/dataset/

In [ ]:
%cd /content/drive/MyDrive/Data_Mining_Project/dataset/
!cp df_train_pro9_TE.csv /content/dataset/
!cp df_test_pro9_TE.csv /content/dataset/

/content/drive/MyDrive/Data_Mining_Project/dataset


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.1 MB/s eta 0:00:00


Now, we start to load data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, f1_score
from sklearn import preprocessing
from sklearn.model_selection import KFold

import xgboost as xgb
from catboost import CatBoostRegressor

In [ ]:
df_train = pd.read_csv('/content/dataset/df_train_pro9_TE.csv')
df_test = pd.read_csv('/content/dataset/df_test_pro9_TE.csv')

In [ ]:
df_train.head()

,month,storey_range,floor_area_sqm,flat_model,lease_commence_date,latitude,longitude,planning_area,resale_price,com_CR_3,...,sec_scl_1,sec_scl_3,shop_1,shop_3,hawker_1,hawker_3,station_1,station_3,pop_m,pop_f
0,200108,2.0,118.0,7,1989,1.369008,103.958697,19,209700.0,1.0,...,4.0,8.0,0.0,8.0,1.0,2.0,0.0,5.0,9885.714286,10110.000000
1,201410,11.0,110.0,4,2003,1.399007,103.906991,20,402300.0,0.0,...,6.0,11.0,4.0,6.0,0.0,0.0,9.0,28.0,10828.000000,11136.000000
2,202009,2.0,112.0,12,2004,1.388348,103.873815,24,351000.0,1.0,...,1.0,10.0,2.0,8.0,0.0,5.0,2.0,13.0,20372.000000,20974.000000
3,200010,8.0,67.0,11,1980,1.318493,103.766702,9,151200.0,0.0,...,2.0,11.0,3.0,11.0,2.0,8.0,1.0,6.0,6275.714286,6828.571429
4,201301,8.0,73.0,7,1985,1.348149,103.742658,3,318600.0,1.0,...,2.0,13.0,1.0,7.0,1.0,4.0,1.0,6.0,8586.250000,8841.250000


In [ ]:
# drop columns
df_train2 = df_train.drop(columns=['sec_scl_1'])
df_train2.head()

df_test2 = df_test.drop(columns=['sec_scl_1'])
df_test2.head()

,month,storey_range,floor_area_sqm,flat_model,lease_commence_date,latitude,longitude,planning_area,com_CR_3,com_CR_5,...,pri_scl_3,sec_scl_3,shop_1,shop_3,hawker_1,hawker_3,station_1,station_3,pop_m,pop_f
0,200401,5.0,94.0,11,1989,1.346581,103.744085,3,1.0,1.0,...,12.0,14.0,1.0,7.0,1.0,5.0,1.0,6.0,8586.250000,8841.250000
1,200111,5.0,122.0,4,1997,1.357618,103.961379,27,1.0,1.0,...,18.0,10.0,0.0,9.0,0.0,3.0,1.0,10.0,32055.000000,33270.000000
2,200207,2.0,67.0,11,1982,1.337804,103.741998,13,1.0,1.0,...,12.0,13.0,4.0,6.0,1.0,5.0,2.0,6.0,8456.000000,8558.000000
3,201504,5.0,82.0,11,1981,1.380084,103.849574,0,0.0,1.0,...,14.0,11.0,3.0,6.0,3.0,12.0,1.0,3.0,8426.000000,9066.000000
4,200404,2.0,117.0,16,1978,1.313960,103.769831,9,0.0,1.0,...,9.0,11.0,3.0,8.0,2.0,9.0,1.0,8.0,6275.714286,6828.571429


In [ ]:
X = df_train2.drop(['resale_price'], axis=1)
y = df_train2['resale_price']

In [ ]:
X.shape

(431732, 29)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(len(X_train))
print(len(X_test))

345385
86347


# Decision Tree Regressor (Hyper-parameters tuning)

In [ ]:
# Define the hyperparameters
params = {
    'max_depth': [20, 25, 30],
    'min_samples_split': [20, 25, 30],
}

# Create a DecisionTreeRegressor instance
dt_rg = DecisionTreeRegressor()

# Create a KFold instance with k folds
# kf = KFold(n_splits=10, shuffle=True, random_state=1)

# Create a GridSearchCV
grid_search = GridSearchCV(estimator=dt_rg, param_grid=params, verbose=3) #cv=kf   5-fold cross validation

# Fit the GridSearchCV on the data
model = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END max_depth=20, min_samples_split=20;, score=0.969 total time=   2.8s
[CV 2/5] END max_depth=20, min_samples_split=20;, score=0.969 total time=   2.9s
[CV 3/5] END max_depth=20, min_samples_split=20;, score=0.968 total time=   2.8s
[CV 4/5] END max_depth=20, min_samples_split=20;, score=0.968 total time=   3.5s
[CV 5/5] END max_depth=20, min_samples_split=20;, score=0.969 total time=   3.2s
[CV 1/5] END max_depth=20, min_samples_split=25;, score=0.969 total time=   2.7s
[CV 2/5] END max_depth=20, min_samples_split=25;, score=0.969 total time=   2.7s
[CV 3/5] END max_depth=20, min_samples_split=25;, score=0.967 total time=   2.7s
[CV 4/5] END max_depth=20, min_samples_split=25;, score=0.968 total time=   3.9s
[CV 5/5] END max_depth=20, min_samples_split=25;, score=0.969 total time=   2.7s
[CV 1/5] END max_depth=20, min_samples_split=30;, score=0.968 total time=   2.6s
[CV 2/5] END max_depth=20, min_samples_split=30;,

In [ ]:
# Print the best hyperparameters found by GridSearchCV
print(model.best_params_) # best: {'max_depth': 25, 'min_samples_split': 25}

# # Predict class labels of test data on the model with the best found parameters
y_pred = model.predict(X_test)
best_mse = mean_squared_error(y_test, y_pred, squared=False)
print(best_mse)

{'max_depth': 25, 'min_samples_split': 20}
22053.55750142075


# Random Forest Regressor (Hyper-parameters tuning)

In [ ]:
# Define the hyperparameters
params = {
    'max_depth': [20, 25, 30],
    'min_samples_split': [20, 25, 30],
}

# Create a RandomForestRegressor instance
rf_rg = RandomForestRegressor(n_estimators=10)

# Create a GridSearchCV instance 
grid_search = GridSearchCV(estimator=rf_rg, param_grid=params, verbose=3) # 5-fold cross validation

# Fit the GridSearchCV instance on the data
model = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END max_depth=20, min_samples_split=20;, score=0.977 total time=  22.2s
[CV 2/5] END max_depth=20, min_samples_split=20;, score=0.978 total time=  19.6s
[CV 3/5] END max_depth=20, min_samples_split=20;, score=0.977 total time=  20.5s
[CV 4/5] END max_depth=20, min_samples_split=20;, score=0.977 total time=  19.5s
[CV 5/5] END max_depth=20, min_samples_split=20;, score=0.978 total time=  20.6s
[CV 1/5] END max_depth=20, min_samples_split=25;, score=0.976 total time=  19.3s
[CV 2/5] END max_depth=20, min_samples_split=25;, score=0.977 total time=  20.4s
[CV 3/5] END max_depth=20, min_samples_split=25;, score=0.976 total time=  19.2s
[CV 4/5] END max_depth=20, min_samples_split=25;, score=0.976 total time=  20.3s
[CV 5/5] END max_depth=20, min_samples_split=25;, score=0.977 total time=  19.6s
[CV 1/5] END max_depth=20, min_samples_split=30;, score=0.976 total time=  20.4s
[CV 2/5] END max_depth=20, min_samples_split=30;,

In [ ]:
# Print the best hyperparameters found by GridSearchCV
print(model.best_params_) 

y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(rmse)

{'max_depth': 25, 'min_samples_split': 20}
18605.81355164614


# Histogram-based Gradient Boosting Regressor (Hyper-parameters tuning)

In [ ]:
# Define the hyperparameters
params = {
    'max_depth': [5, 10, 15, 20, 25],
    'learning_rate': [0.05, 0.1, 0.2],
}

# Create a HistGradientBoostingRegressor instance
hgb_rg = HistGradientBoostingRegressor()

# Create a GridSearchCV instance 
grid_search = GridSearchCV(estimator=hgb_rg, param_grid=params, verbose=3) # 5-fold cross validation

# Fit the GridSearchCV instance on the data
model = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5] END ...learning_rate=0.05, max_depth=5;, score=0.945 total time=   5.8s
[CV 2/5] END ...learning_rate=0.05, max_depth=5;, score=0.948 total time=   4.3s
[CV 3/5] END ...learning_rate=0.05, max_depth=5;, score=0.946 total time=   6.6s
[CV 4/5] END ...learning_rate=0.05, max_depth=5;, score=0.947 total time=   4.3s
[CV 5/5] END ...learning_rate=0.05, max_depth=5;, score=0.947 total time=   5.1s
[CV 1/5] END ..learning_rate=0.05, max_depth=10;, score=0.952 total time=   6.5s
[CV 2/5] END ..learning_rate=0.05, max_depth=10;, score=0.954 total time=   5.1s
[CV 3/5] END ..learning_rate=0.05, max_depth=10;, score=0.952 total time=   7.3s
[CV 4/5] END ..learning_rate=0.05, max_depth=10;, score=0.953 total time=   5.1s
[CV 5/5] END ..learning_rate=0.05, max_depth=10;, score=0.954 total time=   7.4s
[CV 1/5] END ..learning_rate=0.05, max_depth=15;, score=0.952 total time=   5.1s
[CV 2/5] END ..learning_rate=0.05, max_depth=15;

In [ ]:
# Print the best hyperparameters found by GridSearchCV
print(model.best_params_) # best: {'learning_rate': 0.2, 'max_depth': 20}

y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(rmse)

{'learning_rate': 0.2, 'max_depth': 20}
20740.679672418977


# CatBoost with GPU (Hyper-parameters tuning)

In [ ]:
# Define the hyperparameters
params = {
    'depth': [5, 10, 15],
    'learning_rate': [0.05, 0.1, 0.2]
}

# Create a CatBoostRegressor instance
cat_rg = CatBoostRegressor(verbose=0, 
                           task_type="GPU",
                           devices='0:1')

# Create a GridSearchCV instance
grid_search = GridSearchCV(estimator=cat_rg, param_grid=params, verbose=3)  # 5-fold cross validation by default

# Fit the GridSearchCV instance on the data
model = grid_search.fit(X_train, y_train) 

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END .......depth=5, learning_rate=0.05;, score=0.971 total time=   9.7s
[CV 2/5] END .......depth=5, learning_rate=0.05;, score=0.972 total time=   4.8s
[CV 3/5] END .......depth=5, learning_rate=0.05;, score=0.971 total time=   8.1s
[CV 4/5] END .......depth=5, learning_rate=0.05;, score=0.972 total time=   4.9s
[CV 5/5] END .......depth=5, learning_rate=0.05;, score=0.972 total time=   8.2s
[CV 1/5] END ........depth=5, learning_rate=0.1;, score=0.977 total time=   4.9s
[CV 2/5] END ........depth=5, learning_rate=0.1;, score=0.977 total time=   8.0s
[CV 3/5] END ........depth=5, learning_rate=0.1;, score=0.977 total time=   4.8s
[CV 4/5] END ........depth=5, learning_rate=0.1;, score=0.977 total time=   6.0s
[CV 5/5] END ........depth=5, learning_rate=0.1;, score=0.977 total time=   6.0s
[CV 1/5] END ........depth=5, learning_rate=0.2;, score=0.979 total time=   4.8s
[CV 2/5] END ........depth=5, learning_rate=0.2;,

In [ ]:
best_params = model.best_params_
print(best_params)

y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(rmse)

{'depth': 15, 'learning_rate': 0.05}
16100.720404797878


# XGBoost with hyperparameter tuning (with GPU)

In [ ]:
# Define a dictionary of hyperparameters to tune
params = {'max_depth': [10, 15, 20],
          'learning_rate': [0.1, 0.2],
          'reg_alpha': [0, 0.1, 0.5, 1]}

xgb_model = xgb.XGBRegressor(eval_metric=mean_squared_error, tree_method='gpu_hist')


# Define a GridSearchCV object to search for the best combination of hyperparameters
grid_search = GridSearchCV(estimator=xgb_model, param_grid=params, verbose=3)

# Fit the GridSearchCV object to the data
model = grid_search.fit(X_train, y_train)

# Get the best combination of hyperparameters
best_params = model.best_params_
# Print the best hyperparameters found by GridSearchCV
print(best_params)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END learning_rate=0.1, max_depth=10, reg_alpha=0;, score=0.981 total time=   2.9s
[CV 2/5] END learning_rate=0.1, max_depth=10, reg_alpha=0;, score=0.981 total time=   3.2s
[CV 3/5] END learning_rate=0.1, max_depth=10, reg_alpha=0;, score=0.981 total time=   3.9s
[CV 4/5] END learning_rate=0.1, max_depth=10, reg_alpha=0;, score=0.981 total time=   2.7s
[CV 5/5] END learning_rate=0.1, max_depth=10, reg_alpha=0;, score=0.982 total time=   2.7s
[CV 1/5] END learning_rate=0.1, max_depth=10, reg_alpha=0.1;, score=0.981 total time=   3.1s
[CV 2/5] END learning_rate=0.1, max_depth=10, reg_alpha=0.1;, score=0.981 total time=   3.7s
[CV 3/5] END learning_rate=0.1, max_depth=10, reg_alpha=0.1;, score=0.981 total time=   2.8s
[CV 4/5] END learning_rate=0.1, max_depth=10, reg_alpha=0.1;, score=0.981 total time=   2.8s
[CV 5/5] END learning_rate=0.1, max_depth=10, reg_alpha=0.1;, score=0.982 total time=   2.8s
[CV 1/5] END learn

In [ ]:
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(rmse)

16689.253783018816


# Actual training & prediction for the best model

In [ ]:
# Define the hyperparameters
params = {
    'depth': [10, 13, 15],
    'learning_rate': [0.05, 0.1, 0.2]
}

# Create a CatBoostRegressor instance
cat_rg = CatBoostRegressor(verbose=0, 
                           task_type="GPU",
                           devices='0:1')

# Create a GridSearchCV instance
grid_search = GridSearchCV(estimator=cat_rg, param_grid=params, verbose=3)  # 5-fold cross validation by default

# Fit the GridSearchCV instance on the data
model = grid_search.fit(X, y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END ......depth=10, learning_rate=0.05;, score=0.983 total time=  29.2s
[CV 2/5] END ......depth=10, learning_rate=0.05;, score=0.982 total time=  13.5s
[CV 3/5] END ......depth=10, learning_rate=0.05;, score=0.982 total time=  13.6s
[CV 4/5] END ......depth=10, learning_rate=0.05;, score=0.983 total time=  13.5s
[CV 5/5] END ......depth=10, learning_rate=0.05;, score=0.982 total time=  13.7s
[CV 1/5] END .......depth=10, learning_rate=0.1;, score=0.984 total time=  13.6s
[CV 2/5] END .......depth=10, learning_rate=0.1;, score=0.984 total time=  15.8s
[CV 3/5] END .......depth=10, learning_rate=0.1;, score=0.984 total time=  14.2s
[CV 4/5] END .......depth=10, learning_rate=0.1;, score=0.984 total time=  13.7s
[CV 5/5] END .......depth=10, learning_rate=0.1;, score=0.984 total time=  13.8s
[CV 1/5] END .......depth=10, learning_rate=0.2;, score=0.985 total time=  13.0s
[CV 2/5] END .......depth=10, learning_rate=0.2;,

In [ ]:
y_pred_out = model.predict(df_test2)

In [ ]:
len(y_pred_out)

107934

In [ ]:
df_out = pd.DataFrame(y_pred_out, columns = ['Predicted'])
df_out.to_csv('/content/drive/MyDrive/Data_Mining_Project/prediction_result/predicted_TE_wn7.csv', index = True)